<a href="https://colab.research.google.com/github/NicoHerrig95/Dog_Paw/blob/main/MDN_LSTM_VaR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependencies

In [1]:
from tensorflow_probability import distributions as tfd
from tensorflow.keras.layers import Input, Dense, Activation, Concatenate
from tensorflow.keras.models import Model
import keras
import numpy as np
import pandas as pd
import mdn # keras-mdn-layer

In [2]:

# start date and end date of testing period
# shall be the same as in the "initialisation.R" script
test_start_date = "2021-01-01"
test_end_date = "2022-12-31"

In [3]:
#from google.colab import files
#uploaded = files.upload()

In [4]:
#import io
#AAPL = pd.read_csv(io.BytesIO(uploaded['AAPL.csv']))
# Dataset is now stored in a Pandas Dataframe

In [5]:
# generating test data
# data subset for testing purposes
#test_data = AAPL
#test_data["date"] = pd.to_datetime(test_data["date"])
#test_data.drop(columns = ["Unnamed: 0"], inplace = True)  # dropping redundand columns

#AAPL.head()

In [6]:
# data wrangling / train / test



In [7]:
# function for converting data tables to input tables
# adding past returns (t-i for i [0,d]) and generating new data frame

def table_transform(df, rolling_window = 10 ):
    # input data frame for training
    df_con = df.filter(["date","R"]).tail(-1).reset_index(drop = True)

    d = rolling_window # window size
    location = 2 # value for storage location

    # loop for generating columns for past returns
    for i in range(1, d+1):
        df_con.insert(loc = location, column = "R_t-"+str(i), value = np.nan)
        location = location + 1

    del location

    # pasting past returns into columns
    for i in range(0, len(df_con) - d):
        df_con.iloc[i+d, 2:] = df_con.loc[i:(i+d-1), "R"][::-1]


    return df_con

In [8]:
# generating a table for testing with d = 100
#test = table_transform(df = AAPL, rolling_window = 100)
#test = test.tail(-100).reset_index(drop = True) # dropping first 100 rows as they do not include past returns
#test.head()
#test.info()

In [9]:
"""
# defining train set and test set
X = test.drop(columns = ["R", "date"])
Y = test.filter(["R"])


X_train, y_train = X[0:2800], Y[0:2800]
X_test , y_test = X[2800: len(X)], Y[2800: len(Y)]
"""

'\n# defining train set and test set\nX = test.drop(columns = ["R", "date"])\nY = test.filter(["R"])\n\n\nX_train, y_train = X[0:2800], Y[0:2800]\nX_test , y_test = X[2800: len(X)], Y[2800: len(Y)]\n'

" A hitchhikers guide to MDN"

In [10]:
########### DEPENDENCIES ###########

import tensorflow as tf
from tensorflow.keras.layers import Activation

def nnelu(input):
    """ Computes the Non-Negative Exponential Linear Unit
    """
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

tf.keras.utils.get_custom_objects().update({'nnelu': Activation(nnelu)})



"""
LOSS FUNCTION
"""

from tensorflow_probability import distributions as tfd

def slice_parameter_vectors(parameter_vector):
    """ Returns an unpacked list of paramter vectors.
    """
    return [parameter_vector[:,i*components:(i+1)*components] for i in range(no_parameters)]

def gnll_loss(y, parameter_vector):
    """ Computes the mean negative log-likelihood loss of y given the mixture parameters.
    """
    alpha, mu, sigma = slice_parameter_vectors(parameter_vector)  # Unpack parameter vectors

    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha),
        components_distribution=tfd.Normal(
            loc=mu,
            scale=sigma))

    log_likelihood = gm.log_prob(tf.transpose(y))                 # Evaluate log-probability of y

    return -tf.reduce_mean(log_likelihood, axis=-1)


Mixed Density Network Class

In [11]:
# Classe for Mixed Density Network

class MDN(tf.keras.Model):

    def __init__(self, neurons=100, components = 2):
        super(MDN, self).__init__(name="MDN")
        self.neurons = neurons
        self.components = components

        self.h1 = Dense(neurons, activation="relu", name="h1")
        self.h2 = Dense(neurons, activation="relu", name="h2")

        self.alphas = Dense(components, activation="softmax", name="alphas")
        self.mus = Dense(components, name="mus")
        # using nnelu to keep sigma positive
        self.sigmas = Dense(components, activation="nnelu", name="sigmas")
        self.pvec = Concatenate(name="pvec")

    def call(self, inputs):
        x = self.h1(inputs)
        x = self.h2(x)

        alpha_v = self.alphas(x)
        mu_v = self.mus(x)
        sigma_v = self.sigmas(x)

        return self.pvec([alpha_v, mu_v, sigma_v])

In [12]:
"""
PARAMETER DECLARATION FOR NEURAL NETWORK
-> we generally aim at 2 or 3 distributions
2 -> bull / bear
3 -> bull / bear / neutral
"""

no_parameters = 3 # Paramters of the mixture (alpha, mu, sigma)
components = 3 # how many distributions get modelled in the mixture density distribution
neurons = 200 # number of neurons

In [13]:


# Adam optimisation function
opt = tf.optimizers.Adam(1e-3)


# no idea what this is
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau
tensorboard = TensorBoard(log_dir='tbdir/', histogram_freq=0, write_graph=True, write_images=False)
mon = EarlyStopping(monitor='loss', min_delta=0, patience=5, verbose=0, mode='auto')

In [14]:
# defining and compiling the MDN
mdn = MDN(neurons=neurons, components=components)
mdn.compile(loss=gnll_loss, optimizer=opt)

In [15]:
"""
EXAMPLE DATA NON-LINEAR
"""
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


samples = int(1e4) # number of samples

x_data = np.float32(np.random.uniform(-10, 10, (1, samples)))
r_data = np.array([np.random.normal(scale=np.abs(i)) for i in x_data])
y_data = np.float32(np.square(x_data)+r_data*2.0)

x_data2 = np.float32(np.random.uniform(-10, 10, (1, samples)))
r_data2 = np.array([np.random.normal(scale=np.abs(i)) for i in x_data2])
y_data2 = np.float32(-np.square(x_data2)+r_data2*2.0)

x_data = np.concatenate((x_data,x_data2),axis=1).T
y_data = np.concatenate((y_data,y_data2),axis=1).T

#min_max_scaler = MinMaxScaler()
#y_data = min_max_scaler.fit_transform(y_data)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.5, random_state=40, shuffle=True)

In [16]:
mdn.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), epochs=200, callbacks=[mon, tensorboard], batch_size=128, verbose=0)

In [17]:
# example prediction

s = np.linspace(-10,10,int(1e3))[:, np.newaxis].astype(np.float32)
y_pred = mdn.predict(s)
alpha_pred, mu_pred, sigma_pred = slice_parameter_vectors(y_pred)



32/32 [==============================] - 0s 1ms/step


In [21]:
sigma_pred

array([[19.623625, 21.175117, 65.64909 ],
       [19.587692, 21.129663, 65.49252 ],
       [19.551764, 21.084213, 65.33596 ],
       ...,
       [17.086039, 22.626158, 23.207914],
       [17.110931, 22.67573 , 23.280102],
       [17.135832, 22.725302, 23.352293]], dtype=float32)

In [23]:
x_train

array([[ 0.17208439],
       [ 6.9451947 ],
       [-6.3229437 ],
       ...,
       [ 7.696569  ],
       [ 5.911708  ],
       [-7.960221  ]], dtype=float32)

In [19]:
keras.backend.clear_session() # clearing model